# Image Shapes detecting

In [4]:
#!/usr/bin/env python
# coding: utf-8

import cv2
import numpy as np
import pandas as pd
import time
import io
import os
import subprocess
from datetime import datetime
from matplotlib import pyplot as plt

In [ ]:
def showImage(title, img):
    cv2.imshow(title,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
def skeleton(src):
    skel = np.zeros(src.shape,np.uint8)
    size = np.size(src)
    ret,img2 = cv2.threshold(src,127,255,0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    done = False
    
    while( not done):
        eroded = cv2.erode(img2,element)
        temp = cv2.dilate(eroded,element)
        temp = cv2.subtract(img2,temp)
        skel = cv2.bitwise_or(skel,temp)
        img2 = eroded.copy()
    
        zeros = size - cv2.countNonZero(img2)
        if zeros==size:
            done = True

    return skel

In [7]:
image_dir = './video_images/'
for file in os.listdir(image_dir):
    #Original Image
    #file = './CLEANED_VIDEOS/images/screenshot.png'
    img = cv2.imread(image_dir + file)
    showImage('INITIAL IMAGE', img)
        
    #GRAYSCALE
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
    showImage('GRAY', gray)
    
    #EQUALIZED GRAYSCALE
    clahe = cv2.createCLAHE(clipLimit=150.0, tileGridSize=(3,3))
    cl1 = clahe.apply(gray)
    showImage('CLAHE',cl1)
    
    #THRESHOLDED
    thresh = cv2.adaptiveThreshold(cl1, 255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,13,2)
    thresh = cv2.bitwise_not(thresh)
    showImage('THRESH', thresh)
    
    #CLEANING IMAGE, GETTING MOSTLY HORIZ AND VERT LINES
    kernel = np.ones((1,1),np.uint8)
    opening = cv2.erode(thresh, kernel, iterations=1000)
    opening = cv2.dilate(opening, kernel, iterations=1)
    showImage("K1", opening)
    
    kernel = np.array([[0,0,0], [0,1,0], [0,0,0]], np.uint8)
    opening = cv2.erode(opening, kernel, iterations=1)
    showImage("K2", opening)
    
    #HORIZONTAL
    kernel = np.ones((5,1),np.uint8)
    p1 = cv2.erode(opening, kernel, iterations=4)
    p1 = cv2.dilate(p1, kernel, iterations=4)
    
    #VERTICAL
    kernel = np.ones((1,5),np.uint8)
    p2 = cv2.erode(opening, kernel, iterations=3)
    p2 = cv2.dilate(p2, kernel, iterations=3)
    
    #UNION
    union = cv2.bitwise_or(p1, p2)
    
    showImage("UNION", union)
    
    #SKELETONIZED
    skel = skeleton(union)
    showImage("SKEL", skel)
    
    #DILATED SKELETON
    kernel = np.ones((2,1),np.uint8)
    skel1 = cv2.dilate(skel, kernel, iterations=2)
    
    kernel = np.ones((1,2),np.uint8)
    skel2 = cv2.dilate(skel, kernel, iterations=2)
    
    skel3 = cv2.bitwise_or(skel1, skel2)
    
    showImage("SKELD", skel3)
    
    #REMOVE SHORT SEGMENTS
    #min_len = 60
    #black = np.zeros(skel3.shape, np.uint8)
    #r,c = skel3.shape
    #
    #hsegs = []
    #vsegs = []
    #
    #for i in range(r):
    #    count = 0
    #    init = None
    #    end = None
    #    for j in range(c):
    #        if skel3[i][j] != 0:
    #            if count == 0:
    #                init = (j,i)
    #            count += 1
    #        else:
    #            if count >= min_len:
    #                end = (j,i)
    #                cv2.line(black, init, end, (255,255,255))
    #            
    #            count = 0
    #
    #for j in range(c):
    #    count = 0
    #    init = None
    #    end = None
    #    for i in range(r):
    #        if skel3[i][j] != 0:
    #            if count == 0:
    #                init = (j,i)
    #            count += 1
    #        else:
    #            if count >= min_len:
    #                end = (j,i)
    #                cv2.line(black, init, end, (255,255,255))
    #            
    #            count = 0
    #
    #
    #showImage("REDUCED", black)
    
    
    # Find contours
    contours, hierarchy = cv2.findContours(skel3, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    # ## Draw contours
    for cnt in contours:
        approx = []
        epsilon = 0.1*cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        area = cv2.contourArea(cnt)
    
        if (area > 10000 and len(approx) == 4):
            cv2.drawContours(img, [approx], -1, (15, 254, 251), 2)
            showImage('CONTOURS', img)